# Testes de multicolinearidade
- Spearman: Numéricas
- MCC: Categoricas

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from sklearn.metrics import matthews_corrcoef
from itertools import combinations

# Função para calcular a correlação de Spearman para variáveis numéricas
def spearman_corr(df, threshold=0.9):
    corr_matrix = df.corr(method='spearman')
    drop_cols = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                drop_cols.add(corr_matrix.columns[i])
    return drop_cols

# Função para calcular o coeficiente MCC entre variáveis categóricas
def mcc_corr(df, threshold=0.9):
    drop_cols = set()
    for col1, col2 in combinations(df.columns, 2):
        try:
            mcc_value = matthews_corrcoef(df[col1], df[col2])
            if abs(mcc_value) > threshold:
                drop_cols.add(col1)  # Remover col1 se altamente correlacionado
        except ValueError:
            continue  # Ignora colunas com valores únicos
    return drop_cols

# Função principal para identificar e remover colinearidades
def remove_colinearity(df, num_threshold=0.9, cat_threshold=0.9):
    # Separar variáveis numéricas e categóricas
    num_df = df.select_dtypes(include=[np.number])
    cat_df = df.select_dtypes(include=[object, 'category'])
    # Verificar colinearidade em variáveis numéricas
    num_cols_to_drop = spearman_corr(num_df, num_threshold)
    # Verificar colinearidade em variáveis categóricas
    cat_cols_to_drop = mcc_corr(cat_df, cat_threshold)
    # Remover colunas altamente correlacionadas
    df_cleaned = df.drop(columns=list(num_cols_to_drop) + list(cat_cols_to_drop))
    return df_cleaned


In [ ]:
#Aplicando ao dataset amostra
dataset_cleaned = remove_colinearity(amostra, num_threshold=0.9, cat_threshold=0.9)


In [ ]:
# Atualizando join_tables para refletir as colunas selecionadas
join_tables = join_tables[dataset_cleaned.columns]

In [ ]:
# Visualizando o resultado
join_tables.head()